In [1]:
# Part 2 utilities

NEIGHBORS_WITH_DIAG = set(
    (x, y) 
    for x in range(-1, 2) 
    for y in range(-1, 2) 
    if not (x == 0 and y == 0)
)
# diagonal index mapped to shared neighbors between center and diagonal aka buddies
DIAGONAL_GROUPS = {
    (x, y): {(x, 0), (0, y)}
    for x in [-1, 1] for y in [-1, 1]
}

def count_corners(region, data):
    total_corners = 0
    for i, j in sorted(region):
        neighbors = set()
        for x, y in NEIGHBORS_WITH_DIAG:
            a, b = i + x, j + y
            if a in range(len(data)) and b in range(len(data[a])) and data[a][b] == data[i][j]:
                neighbors.add((x, y))

        total_corners += sum(
            any([
                diagonal not in neighbors and buddies.issubset(neighbors), # inner corner on this diagonal
                buddies.isdisjoint(neighbors) # outer corner; ignore diagonal
            ])
            for diagonal, buddies in DIAGONAL_GROUPS.items()
        )
    return total_corners

# Part 1 utilities

NEIGHBORS = [(0, 1), (0, -1), (1, 0), (-1, 0)]

def make_region(i, j, region, data, edge_metric=None):
    if (i, j) in region:
        return set(), 0
    
    region.add((i, j))
    perimeter = 4
    
    neighbors = set((i+x, j+y) for x, y in NEIGHBORS)
    for a, b in neighbors:
        if a in range(len(data)) and b in range(len(data[a])) and data[a][b] == data[i][j]:
            subregion, subperimeter = make_region(a, b, region, data)
            region.update(subregion)
            perimeter += subperimeter - 1
    
    return region, (perimeter if not edge_metric else count_corners(region, data))

Part 1

In [2]:
def pt1(filename, edge_metric=None):
    with open(filename, 'r') as f:
        data = f.read().splitlines()

    visited = set()
    price = 0
    for i in range(len(data)):
        for j in range(len(data)):
            if (i, j) not in visited:
                region, metric = make_region(i, j, set(), data, edge_metric)
                visited.update(region)
                price += len(region) * metric

    return price

pt1('test.txt'), pt1('test2.txt'), pt1('test3.txt'), pt1('input.txt')

(140, 772, 1930, 1374934)

Part 2

In [3]:
(
    pt1('test.txt', edge_metric='sides'),
    pt1('test2.txt', edge_metric='sides'),
    pt1('test3.txt', edge_metric='sides'),
    pt1('input.txt', edge_metric='sides')
)

(80, 436, 1206, 841078)